<div style="text-align: center; line-height: 0; padding-top: 2px;">
  <img src="https://www.quantiaconsulting.com/logos/quantia_logo_orizz.png" alt="Quantia Consulting" style="width: 600px; height: 250px">
</div>

## Getting Started

Let's start importing libraries and creating useful variables 

In [ ]:
%load_ext autotime

In [1]:
from confluent_kafka import avro
from confluent_kafka.avro import AvroProducer
from confluent_kafka.admin import AdminClient, NewTopic


# write a kafka avro producer

Define avro schemas for keys and value to describe cards in a deck

### Key
The key has a single field to contain arbitrary `Name`

### Value
A card is described by an int `Number` and a `Suit`.

**Note**: The card havo only 4 different suits

1) define the key schema

In [2]:
key_schema_str = """
{
  "namespace": "EAP.challenge.avro",
  "type": "record",
  "name": "CardKey",
  "fields": [
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""

key_schema = avro.loads(key_schema_str)

2) define the value schema

In [14]:
value_schema_str = """
{
  "namespace": "EAP.challenge.avro",
  "type": "record",
  "name": "CardValue",
  "fields": [
    {
      "name": "number",
      "type": "int"
    },
    {
      "name": "suittype",
      "type": {
        "type": "enum",
        "name": "Suit",
        "symbols": ["SPADES", "HEARTS", "DIAMONDS", "CLUBS"]
      },
      "doc": "The suit of the card"
    }
  ]
}
"""

value_schema = avro.loads(value_schema_str)

3) create a topic `<surname>-avro-cards` and send in messages such as `SPADES 2`, `HEARTS Q`, `CLUBS K`, ...

In [4]:
servers="broker:29092"

sr_url="http://schema-registry:8081"

topic = 'challenge-avro2'

admin_client = AdminClient({
    "bootstrap.servers": servers
})


In [ ]:
topic_list = []
topic_list.append(NewTopic(topic, 1, 1))
admin_client.create_topics(topic_list)

In [5]:
producerconf = {
        'bootstrap.servers': servers,
        'schema.registry.url': sr_url
    }


In [6]:
ap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

In [7]:
def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))

In [15]:
key = {"name": "Two of spades"}
value = {"number": 2, "suittype": "SPADES"}
ap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

{'number': 2, 'suittype': 'SPADES'}


## Schema Evolution

A system that reads the card data, needs the color of the card for a statistical analysis.

Modify the schema to add the card color, and send new data with the card color.

Do you need to change the topic or the consumer implementation?

In [12]:
value_schema_str = """
{
  "namespace": "EAP.challenge.avro",
  "type": "record",
  "name": "CardValue",
  "fields": [
    {
      "name": "number",
      "type": "int"
    },
    {
      "name": "suittype",
      "type": {
        "type": "enum",
        "name": "Suit",
        "symbols": ["SPADES", "HEARTS", "DIAMONDS", "CLUBS"]
      },
      "doc": "The suit of the card"
    },
    {
      "name": "color",
      "type": {
        "type": "enum",
        "name": "possibletypes",
        "symbols": ["no_color","red", "blue"]
      }, 
      "default": "no_color",
      "doc": "The color of the card"
    }
  ]
}
"""

value_schema = avro.loads(value_schema_str)



In [10]:
def cb(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}"
              .format(msg.topic(), msg.partition(), msg.offset()))



In [13]:

newap = AvroProducer(producerconf, default_key_schema=key_schema, default_value_schema=value_schema)

key = {"name": "Two of spades"}
value = {"number": 2, "suittype": "SPADES", "color":"red"}
newap.produce(topic=topic, value=value, key=key, key_schema=key_schema, value_schema=value_schema)
print(value)

newap.flush(1)

{'number': 2, 'suittype': 'SPADES', 'color': 'red'}


0

##### ![Quantia Tiny Logo](https://www.quantiaconsulting.com/logos/quantia_logo_tiny.png) 2020 Quantia Consulting, srl. All rights reserved.